# 1. Installing YOLOX Libraries

Credit over to Remek Kinas for the following code and his Kaggle notebook enabling YOLOX to be ran without internet: 

Models: https://www.kaggle.com/remekkinas/yolox-cots-models

Code: https://www.kaggle.com/remekkinas/yolox-inference-on-kaggle-for-cots-lb-0-507?scriptVersionId=81625924

In [1]:
import pandas as pd
import numpy as np
import os
import ast
from sklearn.model_selection import GroupKFold
from string import Template
import json
import torch
from shutil import copyfile
import greatbarrierreef
import importlib
import random
import cv2
import matplotlib.pyplot as plt
from PIL import Image
from IPython.display import display

In [2]:
%cp -r /kaggle/input/yolox-cots-models /kaggle/working/
%cd /kaggle/working/yolox-cots-models/yolox-dep

/kaggle/working/yolox-cots-models/yolox-dep


In [3]:
!pip install pip-21.3.1-py3-none-any.whl -f ./ --no-index
!pip install loguru-0.5.3-py3-none-any.whl -f ./ --no-index
!pip install ninja-1.10.2.3-py2.py3-none-manylinux_2_5_x86_64.manylinux1_x86_64.whl -f ./ --no-index
!pip install onnx-1.8.1-cp37-cp37m-manylinux2010_x86_64.whl -f ./ --no-index
!pip install onnxruntime-1.8.0-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl -f ./ --no-index
!pip install onnxoptimizer-0.2.6-cp37-cp37m-manylinux2014_x86_64.whl -f ./ --no-index
!pip install thop-0.0.31.post2005241907-py3-none-any.whl -f ./ --no-index
!pip install tabulate-0.8.9-py3-none-any.whl -f ./ --no-index

Looking in links: ./
Processing ./pip-21.3.1-py3-none-any.whl
pip is already installed with the same version as the provided wheel. Use --force-reinstall to force an installation of the wheel.
Looking in links: ./
Processing ./loguru-0.5.3-py3-none-any.whl
Looking in links: ./
Processing ./ninja-1.10.2.3-py2.py3-none-manylinux_2_5_x86_64.manylinux1_x86_64.whl
Looking in links: ./
Processing ./onnx-1.8.1-cp37-cp37m-manylinux2010_x86_64.whl
Looking in links: ./
Processing ./onnxruntime-1.8.0-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
Looking in links: ./
Processing ./onnxoptimizer-0.2.6-cp37-cp37m-manylinux2014_x86_64.whl
Looking in links: ./
Processing ./thop-0.0.31.post2005241907-py3-none-any.whl
Looking in links: ./
Processing ./tabulate-0.8.9-py3-none-any.whl
tabulate is already installed with the same version as the provided wheel. Use --force-reinstall to force an installation of the wheel.


In [4]:
%cd /kaggle/working/yolox-cots-models/YOLOX
!pip install -r requirements.txt
!pip install -v -e . 

/kaggle/working/yolox-cots-models/YOLOX
Using pip 21.3.1 from /opt/conda/lib/python3.7/site-packages/pip (python 3.7)
Obtaining file:///kaggle/working/yolox-cots-models/YOLOX
  Running command python setup.py egg_info
  running egg_info
  creating /tmp/pip-pip-egg-info-e4l8xrap/yolox.egg-info
  writing /tmp/pip-pip-egg-info-e4l8xrap/yolox.egg-info/PKG-INFO
  writing dependency_links to /tmp/pip-pip-egg-info-e4l8xrap/yolox.egg-info/dependency_links.txt
  writing top-level names to /tmp/pip-pip-egg-info-e4l8xrap/yolox.egg-info/top_level.txt
  writing manifest file '/tmp/pip-pip-egg-info-e4l8xrap/yolox.egg-info/SOURCES.txt'
  reading manifest file '/tmp/pip-pip-egg-info-e4l8xrap/yolox.egg-info/SOURCES.txt'
  adding license file 'LICENSE'
  writing manifest file '/tmp/pip-pip-egg-info-e4l8xrap/yolox.egg-info/SOURCES.txt'
  Preparing metadata (setup.py) ... done
  Running setup.py develop for yolox
    Running command /opt/conda/bin/python3.7 -c 'import io, os, sys, setuptools, tokenize; sy

In [5]:
%cd /kaggle/working/yolox-cots-models/yolox-dep/cocoapi/PythonAPI

!make
!make install
!python setup.py install

/kaggle/working/yolox-cots-models/yolox-dep/cocoapi/PythonAPI
python setup.py build_ext --inplace
running build_ext
cythoning pycocotools/_mask.pyx to pycocotools/_mask.c
/opt/conda/lib/python3.7/site-packages/Cython/Compiler/Main.py:369: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /kaggle/working/yolox-cots-models/yolox-dep/cocoapi/PythonAPI/pycocotools/_mask.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)
building 'pycocotools._mask' extension
creating build
creating build/common
creating build/temp.linux-x86_64-3.7
creating build/temp.linux-x86_64-3.7/pycocotools
gcc -pthread -B /opt/conda/compiler_compat -Wl,--sysroot=/ -Wsign-compare -DNDEBUG -g -fwrapv -O3 -Wall -Wstrict-prototypes -fPIC -I/opt/conda/lib/python3.7/site-packages/numpy/core/include -I../common -I/opt/conda/include/python3.7m -c ../common/maskApi.c -o build/temp.linux-x86_64-3.7/../common/maskApi.o -Wno-cpp -Wno-unused-function -

In [6]:
import pycocotools

In [7]:
N_SPLITS = 5
FOLD = random.randint(0, N_SPLITS - 1)
print(FOLD)

1


In [8]:
%cd /kaggle/working/yolox-cots-models/YOLOX
!mkdir './annotations'
!mkdir './train2017'
!mkdir './val2017'

/kaggle/working/yolox-cots-models/YOLOX


# 2. Splitting Data Into Training/Validation

**EDIT Version 12**: Because we will be cutting out any entries that do not contain any starfish, we will use GroupKFolds to create our folds. 

In [9]:
train = pd.read_csv("/kaggle/input/tensorflow-great-barrier-reef/train.csv")
train['annotations'] = train['annotations'].apply(lambda x: ast.literal_eval(x))
train['box'] = train['annotations'].apply(lambda x: [list(y.values()) for y in x])
train['image_path'] = "video_" + train['video_id'].astype(str) + "/" + train['video_frame'].astype(str) + ".jpg"
train['fold'] = -1
train = train[train['annotations'].str.len() > 0].reset_index(drop = True)

In [10]:
kf = GroupKFold(n_splits=N_SPLITS)
for fold, (train_idx, val_idx) in enumerate(kf.split(train, y = train.video_id.tolist(), groups = train.sequence)):
    train.loc[val_idx, 'fold'] = fold
train['fold'] = train['fold'].astype(int)
train.head()

,video_id,sequence,video_frame,sequence_frame,image_id,annotations,box,image_path,fold
0,0,40258,16,16,0-16,"[{'x': 559, 'y': 213, 'width': 50, 'height': 32}]","[[559, 213, 50, 32]]",video_0/16.jpg,4
1,0,40258,17,17,0-17,"[{'x': 558, 'y': 213, 'width': 50, 'height': 32}]","[[558, 213, 50, 32]]",video_0/17.jpg,4
2,0,40258,18,18,0-18,"[{'x': 557, 'y': 213, 'width': 50, 'height': 32}]","[[557, 213, 50, 32]]",video_0/18.jpg,4
3,0,40258,19,19,0-19,"[{'x': 556, 'y': 214, 'width': 50, 'height': 32}]","[[556, 214, 50, 32]]",video_0/19.jpg,4
4,0,40258,20,20,0-20,"[{'x': 555, 'y': 214, 'width': 50, 'height': 32}]","[[555, 214, 50, 32]]",video_0/20.jpg,4


In [11]:
for i in range(len(train)):
    row = train.loc[i]
    if(row.fold != FOLD):
        copyfile(f'/kaggle/input/tensorflow-great-barrier-reef/train_images/{row.image_path}', f'./train2017/{row.image_id}.jpg')
    else:
        copyfile(f'/kaggle/input/tensorflow-great-barrier-reef/train_images/{row.image_path}', f'./val2017/{row.image_id}.jpg')

# 3. Convert Dataset to YOLOX Format

There are two kinds of evaluators that can be used: the COCO evaluator and the VOC evaluator, both requiring specific formats. Only the COCO evaluator will be made for the time being. 

In [12]:
#COCO:
def datasetToCOCO(dataset):
    annotation = 0
    json = {
        "info": [],
        "licenses": [],
        "categories": [],
        "images": [],
        "annotations": []
    }
    info = {
        "year": "2021",
        "version": "1",
        "description": "COTS dataset - COCO format",
        "contributor": "",
        "url": "https://kaggle.com",
        "date_created": "2021-11-30T15:01:26+00:00"
    }
    json['info'].append(info)
    licenses = {
        "id": 1,
        "url": "",
        "name": "Unknown"
    }
    json['licenses'].append(licenses)
    categories = {
        "id": 0, 
        "name": "starfish", 
        "supercategory": "none"
    }
    json["categories"].append(categories)
    for row in dataset.itertuples():
        images = {
            "id": row[0],
            "license": 1,
            "file_name": row.image_id + '.jpg',
            "height": 720,
            "width": 1280,
            "date_captured": "2021-11-30T15:01:26+00:00"
        }
        json['images'].append(images)
        boxes = row.box
        for box in boxes:
            width = box[2]
            height = box[3]
            if (box[0] + box[2] > 1280):
                width = 1280 - box[0] 
            if (box[1] + box[3] > 720):
                height = 720 - box[1] 
            annotations = {
                "id": annotation,
                "image_id": row[0],
                "category_id": 0,
                "bbox": [box[0], box[1], width, height],
                "area": box[2] * box[3],
                "segmentation": [],
                "iscrowd": 0
            }
            annotation += 1
            json['annotations'].append(annotations)
    return json

In [13]:
train_coco = datasetToCOCO(train[train['fold'] != FOLD])
with open(f"./annotations/train.json", 'w') as f:
    output_json = json.dumps(train_coco)
    f.write(output_json)
val_coco = datasetToCOCO(train[train['fold'] == FOLD])
with open(f"./annotations/val.json", 'w') as f:
    output_json = json.dumps(val_coco)
    f.write(output_json)

# 4. Apply Evaluator

There are multiple kinds of YOLOX models, but for the time being, we will just focus on YOLOX-s.

In [14]:
template = '''
#!/usr/bin/env python3
# -*- coding:utf-8 -*-
# Copyright (c) Megvii, Inc. and its affiliates.

import os

from yolox.exp import Exp as MyExp


class Exp(MyExp):
    def __init__(self):
        super(Exp, self).__init__()
        self.num_classes = 1
        self.depth = 0.33
        self.width = 0.50
        self.exp_name = os.path.split(os.path.realpath(__file__))[1].split(".")[0]
        self.data_dir = "/kaggle/working/yolox-cots-models/YOLOX/"
        self.train_ann = 'train.json'
        self.val_ann = 'val.json'
        self.max_epoch = $max_epoch
        #self.eval_interval = 1
        #self.data_num_workers = 2
        #self.input_size = (960, 960)
        #self.test_size = (960, 960)
        #self.no_aug_epochs = 2
        #self.mosaic_scale = (0.5, 1.5)
        #self.random_size = (10, 20)
'''

In [15]:
pipeline = Template(template).substitute(max_epoch = 20)
with open('cots_config.py', 'w') as f:
    f.write(pipeline)

In [16]:
#VERSION 14: Added voc_cls to see if any differences arise.
voc_cls = '''
VOC_CLASSES = (
  "starfish",
)
'''
with open('/kaggle/working/yolox-cots-models/YOLOX/yolox/data/datasets/voc_classes.py', 'w') as f:
    f.write(voc_cls)

coco_cls = '''
COCO_CLASSES = (
  "starfish",
)
'''
with open('/kaggle/working/yolox-cots-models/YOLOX/yolox/data/datasets/coco_classes.py', 'w') as f:
    f.write(coco_cls)

# 5. Weights

And now to import the weights file for YOLOX-s.

In [17]:
MODEL = '/kaggle/working/yolox-cots-models/yolox_s.pth'

# 6. Training

In [18]:
!cp /kaggle/working/yolox-cots-models/YOLOX/tools/train.py ./

In [19]:
!python train.py -f cots_config.py -d 1 -b 32 --fp16 -o -c {MODEL}

2022-01-11 22:30:07 | INFO     | yolox.core.trainer:126 - args: Namespace(batch_size=32, cache=False, ckpt='/kaggle/working/yolox-cots-models/yolox_s.pth', devices=1, dist_backend='nccl', dist_url=None, exp_file='cots_config.py', experiment_name='cots_config', fp16=True, machine_rank=0, name=None, num_machines=1, occupy=True, opts=[], resume=False, start_epoch=None)
2022-01-11 22:30:07 | INFO     | yolox.core.trainer:127 - exp value:
╒══════════════════╤════════════════════════════════════════════╕
│ keys             │ values                                     │
╞══════════════════╪════════════════════════════════════════════╡
│ seed             │ None                                       │
├──────────────────┼────────────────────────────────────────────┤
│ output_dir       │ './YOLOX_outputs'                          │
├──────────────────┼────────────────────────────────────────────┤
│ print_interval   │ 10                                         │
├──────────────────┼──────────────

# 7. Evaluation

The following was taken from Remek Kina's excellent training pipeline notebook: https://www.kaggle.com/remekkinas/yolox-training-pipeline-cots-dataset-lb-0-507

In [20]:
from yolox.utils import postprocess
from yolox.data.data_augment import ValTransform

In [21]:
current_exp = importlib.import_module('cots_config')
exp = current_exp.Exp()
test_size = (640, 640)
num_classes = 1
confthre = 0.01
nmsthre = 0.65
model = exp.get_model()
model.cuda()
model.eval()
ckpt = torch.load("./YOLOX_outputs/cots_config/best_ckpt.pth", map_location="cpu")
model.load_state_dict(ckpt["model"])

<All keys matched successfully>

In [22]:
%cd /kaggle/working/
def yolox_inference(img, model, test_size): 
    bboxes = []
    bbclasses = []
    scores = []
    
    preproc = ValTransform(legacy = False)

    tensor_img, _ = preproc(img, None, test_size)
    tensor_img = torch.from_numpy(tensor_img).unsqueeze(0)
    tensor_img = tensor_img.float()
    tensor_img = tensor_img.cuda()

    with torch.no_grad():
        outputs = model(tensor_img)
        outputs = postprocess(
                    outputs, num_classes, confthre,
                    nmsthre, class_agnostic=True
                )

    if outputs[0] is None:
        return [], [], []
    
    outputs = outputs[0].cpu()
    bboxes = outputs[:, 0:4]

    bboxes /= min(test_size[0] / img.shape[0], test_size[1] / img.shape[1])
    bbclasses = outputs[:, 6]
    scores = outputs[:, 4] * outputs[:, 5]
    
    return bboxes, bbclasses, scores

/kaggle/working


In [23]:
env = greatbarrierreef.make_env()   # initialize the environment
iter_test = env.iter_test()    # an iterator which loops over the test set and sample submission

In [24]:
submission_dict = {
    'id': [],
    'prediction_string': [],
}

for (image_np, sample_prediction_df) in iter_test:
 
    bboxes, bbclasses, scores = yolox_inference(image_np, model, test_size)
    
    predictions = []
    for i in range(len(bboxes)):
        box = bboxes[i]
        cls_id = int(bbclasses[i])
        score = scores[i]
        if score < confthre:
            continue
        x_min = int(box[0])
        y_min = int(box[1])
        x_max = int(box[2])
        y_max = int(box[3])
        
        bbox_width = x_max - x_min
        bbox_height = y_max - y_min
        
        predictions.append('{:.2f} {} {} {} {}'.format(score, x_min, y_min, bbox_width, bbox_height))
    
    prediction_str = ' '.join(predictions)
    sample_prediction_df['annotations'] = prediction_str
    env.predict(sample_prediction_df)

    print('Prediction:', prediction_str)

This version of the API is not optimized and should not be used to estimate the runtime of your code on the hidden test set.
Prediction: 0.04 72 496 38 39 0.02 65 264 57 35
Prediction: 0.06 60 512 46 38
Prediction: 0.02 48 518 37 36


In [25]:
sub_df = pd.read_csv('submission.csv')
sub_df.head()

,index,annotations
0,0,0.04 72 496 38 39 0.02 65 264 57 35
1,1,0.06 60 512 46 38
2,2,0.02 48 518 37 36
